In [1]:
!nvidia-smi

Fri Jul 16 10:56:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    37W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
!git clone https://github.com/ibrahim-elsawy/dstilPegasus
# !pip install -r /content/konafa/colab_req.txt
!pip install datasets
#!pip install torch==1.7.1
!pip install transformers==4.3.0rc1
!pip install sentencepiece 
!pip install catalyst==20.6
!pip install rouge-score
!pip install sacrebleu
!pip install torch

Cloning into 'dstilPegasus'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 427 (delta 13), reused 2 (delta 2), pack-reused 400
Receiving objects: 100% (427/427), 156.12 KiB | 4.34 MiB/s, done.
Resolving deltas: 100% (271/271), done.
     |████████████████████████████████| 262 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 243 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 67.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 52.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 368 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 831.4 M

In [3]:
os.chdir('/home/jupyter/dstilPegasus')
import torch
import transformers
from catalyst import dl
from src.runners import DistilMLMRunner
from src.runners import DistilMLMRunnerFT
# from src.models import DistilpegasusStudentModel, PegasusForMLM #BertForMLM  DistilbertStudentModel
from catalyst.core import MetricAggregationCallback
from torch.utils.data import DataLoader
from src.callbacks import (
    CosineLossCallback,
    KLDivLossCallback,
    MaskedLanguageModelCallback,
    MSELossCallback,
    PerplexityMetricCallbackDistillation,
    SmoothingLossCallback,
    CrossentropylossCallback
)
from utils.rouge import calc_generative_metrics 
import pandas as pd
from typing import Iterable, Union

# import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
# import transformers
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorForLanguageModeling
import numpy as np
from transformers.data.data_collator import default_data_collator
from torch import nn
from typing import Dict, List
from datasets import load_dataset
import datasets
from datasets import Dataset

In [4]:
os.chdir("/home/jupyter/")

In [5]:
!pwd

/home/jupyter


In [6]:
bucket_name = "bucket-nlpgp-1"
project_id = "smart-impact-314500"

In [14]:
!mkdir pandas

In [15]:
!gsutil cp gs://{bucket_name}/pandas/dataset.arrow /home/jupyter/pandas/

Copying gs://bucket-nlpgp-1/pandas/dataset.arrow...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][  2.2 GiB/  2.2 GiB]      0.0 B/s                                   
Operation completed over 1 objects/2.2 GiB.                                      


In [18]:
!gsutil cp gs://{bucket_name}/pandas/dataset_info.json /home/jupyter/pandas/

Copying gs://bucket-nlpgp-1/pandas/dataset_info.json...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


In [19]:
!gsutil cp gs://{bucket_name}/pandas/state.json /home/jupyter/pandas/

Copying gs://bucket-nlpgp-1/pandas/state.json...
/ [1 files][  387.0 B/  387.0 B]                                                
Operation completed over 1 objects/387.0 B.                                      


In [20]:
!ls

dstilPegasus  gcp1.ipynb  nltk_data  pandas  src  tutorials


In [21]:
!ls pandas

dataset.arrow  dataset_info.json  state.json


In [22]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
train_dataset = datasets.load_from_disk("/home/jupyter/pandas")
teacher = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum",max_length = 1000,max_position_embeddings=1000).to('cuda')#

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [23]:
import datasets
import warnings
import torch
from torch import nn
from typing import Optional, Tuple, List, Union
from transformers import PegasusModel, PegasusConfig, PegasusForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers import SummarizationPipeline
students_config_book = {
    '2': PegasusConfig(encoder_layers=2, decoder_layers=2),
    '4': PegasusConfig(encoder_layers=4, decoder_layers=4),
    '6': PegasusConfig(encoder_layers=6, decoder_layers=6),
    '8': PegasusConfig(encoder_layers=8, decoder_layers=8),
    '10': PegasusConfig(encoder_layers=10, decoder_layers=10),
    '12': PegasusConfig(encoder_layers=12, decoder_layers=12),
    '16': PegasusConfig(encoder_layers=16, decoder_layers=16)
}


LAYERS_TO_COPY = {
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15],
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },
}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15]},
}


def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(
        layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

# Copied from transformers.models.bart.modeling_bart.shift_tokens_right


def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))


def get_layers_to_supervise(n_student, n_teacher) -> List[int]:
    """Used or the --supervise_forward kwarg"""
    if n_student > n_teacher:
        raise ValueError(
            f"Cannot perform intermediate supervision for student {n_student} > teacher {n_teacher}")
    elif n_teacher == n_student:
        return list(range(n_teacher))
    elif n_student == 1:
        return [n_teacher - 1]
    else:
        return LAYERS_TO_SUPERVISE[n_teacher][n_student]


def create_student_with_configuration(teacher,
                                      e=None,
                                      d=None,
                                      copy_first_teacher_layers = False,
                                      save_path='./student'):

    teacher.eval()
    teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
    init_kwargs = teacher.config.to_diff_dict()
    if e is None:
        e = teacher_e
    if d is None:
        d = teacher_d
    init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    student_cfg = teacher.config_class(**init_kwargs)
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    # every student key should have a teacher keys.
    assert info.missing_keys == [], info.missing_keys

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    copy_layers(teacher.model.encoder.layers,
                student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers,
                student.model.decoder.layers, d_layers_to_copy)

    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    #student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student


In [24]:
student = create_student_with_configuration(teacher,
                                     e=16,
                                     d=4,
                                     copy_first_teacher_layers = False,
                                     save_path='./student').to('cuda')
# student = teacher
model = torch.nn.ModuleDict({"student": student})
#checkpoint = torch.load('trained_student_6_4_noTA.pt', map_location=torch.device('cuda'))
#student.load_state_dict(checkpoint['model_state_dict'])
del teacher

In [25]:
torch.cuda.empty_cache()
!nvidia-smi

Fri Jul 16 11:10:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    47W / 300W |   2763MiB / 16160MiB |     51%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
for param in student.model.shared.parameters():
  param.requires_grad = False
for param in student.model.encoder.parameters():
  param.requires_grad = False
# for param in student.model.encoder.layers[-2:].parameters():
#   param.requires_grad = True
# for param in student.model.encoder.embed_tokens.parameters():
#   param.requires_grad = False
# for param in student.model.encoder.embed_positions.parameters():
#   param.requires_grad = False
# for param in student.model.decoder.embed_tokens.parameters():
#   param.requires_grad = False
# for param in student.model.decoder.embed_positions.parameters():
#   param.requires_grad = False
# for param in student.model.shared.parameters():
#   param.requires_grad = False
# for param in student.model.encoder.embed_tokens.parameters():
#   param.requires_grad = False
# for param in student.model.encoder.embed_positions.parameters():
#   param.requires_grad = False
# for param in student.model.decoder.parameters():
#   param.requires_grad = False
#for param in student.model.encoder.layers[-2:].parameters():
  #param.requires_grad = True

In [27]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5,weight_decay = 0.0)

In [28]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [29]:
train_dataloader = DataLoader(train_dataset, batch_size=8)
loaders = {"train": train_dataloader}

In [30]:
callbacks = {
    #"masked_lm_loss": MaskedLanguageModelCallback(),
    #"mse_loss": MSELossCallback(),
    "cross_ent_loss":CrossentropylossCallback(),
    #"cosine_loss": CosineLossCallback(),
    #"kl_div_loss": KLDivLossCallback(),
    #"label_smooth_loss": SmoothingLossCallback(),
    "loss": MetricAggregationCallback(
        prefix="loss",
        mode="weighted_sum",
        metrics={
            #"cosine_loss": 3.0,
            # "masked_lm_loss": 1.0,
            #"kl_div_loss": 0.8,
            "cross_ent_loss" : 1,
            #"mse_loss": 3.0,
            #"label_smooth_loss": .2,
        }
    ),
    "optimizer": dl.OptimizerCallback(),
    # "perplexity": PerplexityMetricCallbackDistillation()
}

In [31]:
!pwd

/home/jupyter


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import pickle
import time
runner = DistilMLMRunnerFT(device=torch.device("cuda"))
runner.device = torch.device("cuda")
#optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
try:
    runner.train(
        model=model,
        optimizer=optimizer, 
        loaders=loaders,
        verbose=True,
        #check=True,
        callbacks=callbacks,
        num_epochs = 2,
        #check = True,
        # engine='cpu',
    )
    torch.save({ 
        'epoch': runner.epoch,
        'model_state_dict': student.state_dict(),
        'optimizer_state_dict': runner.optimizer.state_dict(),
        }, 'trained_student.pt')
    # model_name = "google/pegasus-xsum"
    # tokenizer = PegasusTokenizer.from_pretrained(model_name)
    # #dict_scores = {"rouge1":[], "rouge2":[], "rougeL":[], "rougeLsum":[],"all_time" : []}
    # a_file = open("rouge_scores_6_4_noTA.pkl", "rb")
    # dict_scores = pickle.load(a_file)
    # a_file.close()
    # for i,batch in enumerate(valid_dataloader):
    #   labels = tokenizer.batch_decode(batch['decode_ids'],skip_special_tokens=True)
    #   time_before = time.time()
    #   pred_ids = student.generate(input_ids = batch['input_ids'],attention_mask = batch['attention_mask'],max_length=60,use_cache=True )
    #   time_after = time.time()
    #   pred = tokenizer.batch_decode(pred_ids,skip_special_tokens=True)
    #   dict_scores['all_time'].append((time_after-time_before)/batch["input_ids"].shape[0])
    #   scores = calc_generative_metrics(pred,labels)
    #   dict_scores['rouge1'].append(scores['rouge1'])
    #   dict_scores['rouge2'].append(scores['rouge2'])
    #   dict_scores['rougeL'].append(scores['rougeL'])
    #   dict_scores['rougeLsum'].append(scores['rougeLsum'])
    # a_file = open("rouge_scores_6_4_noTA.pkl", "wb")
    # pickle.dump(dict_scores, a_file)
    # a_file.close()
except RuntimeError as e:
    torch.save({ 
        'epoch': runner.epoch,
        'model_state_dict': student.state_dict(),
        'optimizer_state_dict': runner.optimizer.state_dict(),
        'loss': runner.epoch_metrics,
        }, 'trained_student_error.pt')
    print(f"\n{e}\n Student Model is saved................")

/opt/conda/lib/python3.7/site-packages/catalyst/dl/experiment/experiment.py:164: UserWarning:

Attention, there is only one dataloader - train



1/2 * Epoch (train):   0% 0/50000 [00:00<?, ?it/s]train
1/2 * Epoch (train):   0% 1/50000 [00:02<30:00:54,  2.16s/it, cross_ent_loss=10.049, loss=10.049]train
1/2 * Epoch (train):   0% 2/50000 [00:02<17:49:59,  1.28s/it, cross_ent_loss=10.058, loss=10.058]train
1/2 * Epoch (train):   0% 3/50000 [00:03<14:08:02,  1.02s/it, cross_ent_loss=9.882, loss=9.882]  train
1/2 * Epoch (train):   0% 4/50000 [00:04<12:17:13,  1.13it/s, cross_ent_loss=10.029, loss=10.029]train
1/2 * Epoch (train):   0% 5/50000 [00:04<11:19:03,  1.23it/s, cross_ent_loss=10.246, loss=10.246]train
1/2 * Epoch (train):   0% 6/50000 [00:05<10:44:07,  1.29it/s, cross_ent_loss=9.702, loss=9.702]  train
1/2 * Epoch (train):   0% 7/50000 [00:06<10:21:22,  1.34it/s, cross_ent_loss=9.998, loss=9.998]train
1/2 * Epoch (train):   0% 8/50000 [00:07<10:14:09,  1.36it/s, cross_ent_loss=9.710, loss=9.710]train
1/2 * Epoch (train):   0% 9/50000 [00:07<10:01:29,  1.39it/s, cross_ent_loss=10.161, loss=10.161]train
1/2 * Epoch (train): 

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=runs 